In [1]:
# --- repo setup ---
import os, sys, subprocess

REPO_URL = "https://github.com/Wmeesena/Safe_Start.git"
REPO_DIR = "/content/Safe_Start/MAGIC"

if not os.path.isdir(REPO_DIR):
    subprocess.run(["git", "clone", REPO_URL], check=True)

# if you use Git LFS, uncomment these two lines:
# subprocess.run(["apt-get", "-y", "install", "git-lfs"], check=True)
# subprocess.run(["git", "-C", REPO_DIR, "lfs", "pull"], check=True)

# go to repo root and make imports like `from src...` work
os.chdir(REPO_DIR)
sys.path.append(REPO_DIR)

# optional: pull submodules if you have any
# subprocess.run(["git", "-C", REPO_DIR, "submodule", "update", "--init", "--recursive"], check=True)

print("CWD:", os.getcwd())
%pip install ucimlrepo
%pip install -U ipython

CWD: /content/Safe_Start/MAGIC
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.6.0 which is incompatible.


In [7]:
!ls

experiments.ipynb  src


In [6]:
# test

In [9]:
%git add .

UsageError: Line magic function `%git` not found.


# Setup

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import torch
from src.models.mlp import MLP
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
from src.train import pretrain, evaluate_avg_accuracy, evaluate_robust, joint_train


%load_ext autoreload
%autoreload 2


# Data

In [ ]:
SIGMA = 40.0
SAMPLES_EVAL = 5000
GAMMA = 1000.

In [ ]:
magic_gamma_telescope = fetch_ucirepo(id=159)

# data (as pandas dataframes)
X_df = magic_gamma_telescope.data.features
y_df = magic_gamma_telescope.data.targets

y_df = (y_df == 'g').astype(float)

X_train_df = X_df.sample(n=15000, random_state=41)
X_test_df = X_df.drop(X_train_df.index)

y_train_df = y_df.loc[X_train_df.index]
y_test_df = y_df.drop(y_train_df.index)


# test positive only/negative only robustness

X_test_positive_df = X_test_df.loc[y_test_df["class"] == 1.0]
X_test_negative_df = X_test_df.loc[y_test_df["class"] == 0.0]
y_test_positive_df = y_test_df.loc[y_test_df["class"] == 1.0]
y_test_negative_df =y_test_df.loc[y_test_df["class"] == 0.0]


# train positive only/negative only robustness

X_train_positive_df = X_train_df.loc[y_train_df["class"] == 1.0]
X_train_negative_df = X_train_df.loc[y_train_df["class"] == 0.0]
y_train_positive_df = y_train_df.loc[y_train_df["class"] == 1.0]
y_train_negative_df =y_train_df.loc[y_train_df["class"] == 0.0]


In [ ]:
X_train = torch.tensor(X_train_df.values, dtype=torch.float32)
X_test = torch.tensor(X_test_df.values, dtype=torch.float32)

y_train = torch.tensor(y_train_df.values, dtype=torch.float32)
y_test = torch.tensor(y_test_df.values, dtype=torch.float32)


# train positive only/negative only robustness

X_train_positive = torch.tensor(X_train_positive_df.values, dtype=torch.float32)
X_train_negative = torch.tensor(X_train_negative_df.values, dtype=torch.float32)
y_train_positive = torch.tensor(y_train_positive_df.values, dtype=torch.float32)
y_train_negative = torch.tensor(y_train_negative_df.values, dtype=torch.float32)


# test positive only/negative only robustness
X_test_positive = torch.tensor(X_test_positive_df.values, dtype=torch.float32)
X_test_negative = torch.tensor(X_test_negative_df.values, dtype=torch.float32)
y_test_positive = torch.tensor(y_test_positive_df.values, dtype=torch.float32)
y_test_negative = torch.tensor(y_test_negative_df.values, dtype=torch.float32)


In [ ]:
model = MLP(input_size=X_df.shape[1])
pretrained_model = pretrain(X_train,y_train, model, num_epochs=1000)

In [ ]:
pretrain_acc = evaluate_avg_accuracy(X_train, y_train,
                                        pretrained_model)
RA_pretrain, CRA_pretrain = evaluate_robust(X_test,
                                        y_test, pretrained_model,
                                        num_samples = SAMPLES_EVAL,
                                        sigma = SIGMA)

Accuracy: 83.51%
Robust Accuracy: 42.85989%
Conditional Robust Accuracy: 38.79281%


In [ ]:
naive_model = joint_train(X_train, y_train,
                             X_train_positive, y_train_positive,
                             pretrained_model, num_epochs = 100,
                              gamma = GAMMA, num_samples = 100,
                              sigma = SIGMA)
safe_model = joint_train(X_train, y_train, X_train_positive,
                        y_train_positive, pretrained_model,
                         num_epochs = 100, gamma = GAMMA,
                         num_samples = 100, sigma = SIGMA, IF_SAFE=True)

safe_negative_model = joint_train(X_train, y_train, X_train_positive,
                        y_train_positive, pretrained_model,
                         num_epochs = 100, gamma = GAMMA,
                         num_samples = 100, sigma = SIGMA, IF_SAFE=True, SAFE_BIAS = -1000)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [ ]:
# naive continue training
naive_acc = evaluate_avg_accuracy(X_test,
                                         y_test,
                                        naive_model)
RA_naive, CRA_naive =  evaluate_robust(X_test_positive,
                                                y_test_positive,
                                                naive_model,
                                                num_samples = SAMPLES_EVAL,
                                                 sigma = SIGMA)

# safe continue training

safe_acc = evaluate_avg_accuracy(X_test,
                                    y_test,
                                    safe_model)
RA_safe, CRA_safe = evaluate_robust(X_test_positive,
                                                 y_test_positive,
                                                 safe_model,
                                                 num_samples = SAMPLES_EVAL, sigma = SIGMA)

Accuracy: 63.48%
Robust Accuracy: 96.08697%
Conditional Robust Accuracy: 96.08697%
Accuracy: 68.11%
Robust Accuracy: 98.86528%
Conditional Robust Accuracy: 99.03251%


In [ ]:
safe_model_1 = joint_train(X_train, y_train,
                            X_train_positive, y_train_positive,
                            pretrained_model, num_epochs = 1000,
                            gamma = 10/QUANTILE, num_samples = 10, sigma = SIGMA,
                            IF_SAFE=True)
safe1_acc = evaluate_avg_accuracy(X_test,
                                    y_test,
                                    safe_model_1)
safe1_positive_acc, safe_var = evaluate_robust(X_test_positive,
                                                 y_test_positive,
                                                 safe_model_1,
                                                 num_samples = 100000, sigma = SIGMA,
                                                 quantile = QUANTILE)

Accuracy: 64.33%


: 

In [ ]:
naive_model_1 = joint_train(X_train, y_train,
                         X_train_positive, y_train_positive,
                         pretrained_model, num_epochs = 1000,
                         gamma = 10/QUANTILE, num_samples = 10, sigma = SIGMA)
naive1_acc = evaluate_avg_accuracy(X_test,
                                    y_test,
                                    naive_model_1)
naive1_positive_acc, safe_var = evaluate_robust(X_test_positive,
                                                 y_test_positive,
                                                 naive_model_1,
                                                 num_samples = 100000, sigma = SIGMA,
                                                 quantile = QUANTILE)

Accuracy: 63.48%
Mean Accuracy: 99.99581%
1e-06 Quantile Accuracy: 99.51000%


In [ ]:
safe_model_1